In [1]:
import os, sys, subprocess
import glob

`base_dir`: where the logs are stored.<br>
`identifier`: all dirs with this in their names will be selected.<br>
`excluders`: all dirs with these in their names will be excluded.<br>
`job_num`: check one job num or all. (`*`).<br>
`err`: error to select files based on.

In [21]:
base_dir = '/work/submit/lavezzo/SUEP/logs/'
identifiers = ['inf', '20UL17']
excluders = ['inf_data']
err = '"A process in the process pool was terminated abruptly"'
job_num = '*'      # set to * for any

In [22]:
dirs = [name for name in os.listdir(base_dir) if all([i in name for i in identifiers])]
dirs = [name for name in dirs if not any([excl in name for excl in excluders])]
# these are the directories selected to be analyzed
dirs

['jobs_inf_2016_2017_QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8+RunIISummer20UL17MiniAODv2-106X_mc2017_realistic_v9-v1+MINIAODSIM',
 'jobs_inf_2016_2017_QCD_HT50to100_TuneCP5_PSWeights_13TeV-madgraph-pythia8+RunIISummer20UL17MiniAODv2-106X_mc2017_realistic_v9-v1+MINIAODSIM',
 'jobs_inf_2016_2017_QCD_Pt_50to80_TuneCP5_13TeV_pythia8+RunIISummer20UL17MiniAODv2-106X_mc2017_realistic_v9-v1+MINIAODSIM',
 'jobs_inf_2016_2017_QCD_HT1500to2000_TuneCP5_PSWeights_13TeV-madgraph-pythia8+RunIISummer20UL17MiniAODv2-106X_mc2017_realistic_v9-v1+MINIAODSIM',
 'jobs_inf_2016_2017_QCD_Pt_120to170_TuneCP5_13TeV_pythia8+RunIISummer20UL17MiniAODv2-106X_mc2017_realistic_v9-v1+MINIAODSIM',
 'jobs_inf_2016_2017_QCD_HT300to500_TuneCP5_PSWeights_13TeV-madgraph-pythia8+RunIISummer20UL17MiniAODv2-106X_mc2017_realistic_v9-v1+MINIAODSIM',
 'jobs_inf_2016_2017_QCD_Pt_170to300_TuneCP5_13TeV_pythia8+RunIISummer20UL17MiniAODv2-106X_mc2017_realistic_v9-v1+MINIAODSIM',
 'jobs_inf_2016_2017_QCD_Pt_1400to1800_TuneCP5_13TeV_pyth

In [8]:
for d in dirs:
    from_dir = d
    d = d.split("jobs_inf_")[-1]
    to_dir = 'jobs_inf_2016_2017_' + d
    os.system("mv " + base_dir + from_dir + " " + base_dir + to_dir)

In [9]:
# How many files have this error
for i,dir_name in enumerate(dirs):
    print(dir_name)
    logs = glob.glob(base_dir+dir_name+"/*.err")
    logs = [int(l.split('/')[-1].split('.')[0]) for l in logs]
    logs = list(set(logs))
    logs.sort()
    last_job = logs[-1]
    job_num = str(last_job)
    error_files = os.popen('grep '+ err + ' ' + base_dir + dir_name +'/' + job_num + '.*.err').readlines()
    print(len(error_files))

jobs_inf_2016_2017_2016_2017_QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8+RunIISummer20UL16MiniAODv2-106X_mcRun2_asymptotic_v17-v1+MINIAODSIM


IndexError: list index out of range

`WARNING`: this cell will **delete** files.<br>
This will find all files with the specific error in the .err, and delete the correspdoning HDF5 file.<br>
This might need to be adjusted based on the error / log files outputs. First run it dry to make sure it works.

In [12]:
base_out_dir = '/mnt/T3_US_MIT/hadoop/scratch/lavezzo/SUEP/inf/'
dry = False

for i, dir_name in enumerate(dirs):
    print(dir_name)
    # find .err files with error
    # logs = glob.glob(base_dir+dir_name+"/*.err")
    # logs = [int(l.split('/')[-1].split('.')[0]) for l in logs]
    # logs = list(set(logs))
    # logs.sort()
    # last_job = logs[-1]
    # job_num = str(last_job)
    error_files = os.popen('grep '+ err + ' ' + base_dir + dir_name +'/' + job_num + '.*.err').readlines()
    print(len(error_files))
    for f in error_files:
        
        # find corresponding .out files
        outfile = f.split(":")[0][:-3]+"out"
        
        # grab name of .root file
        file_hdf = os.popen('grep "xrdcp root" ' + outfile).readlines()[0].split("/")[-1].split(" ")[0]
        file_hdf = file_hdf.split(".root")[0]+".hdf5"
        
        # run it dry to make sure you are deleting what you want to delete
        if dry:
            print(['rm',base_out_dir+dir_name.split('jobs_inf_')[-1]+"/"+file_hdf])
            break
            
        # delete the hdf5 file
        else:
            try: 
                _ = subprocess.run(['rm',base_out_dir+dir_name.split('jobs_inf_')[-1]+"/"+file_hdf],
                              stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            except:
                continue

jobs_inf_2016_2017_2016_2017_QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8+RunIISummer20UL16MiniAODv2-106X_mcRun2_asymptotic_v17-v1+MINIAODSIM
0
jobs_inf_2016_2017_2016_2017_QCD_HT50to100_TuneCP5_PSWeights_13TeV-madgraphMLM-pythia8+RunIISummer20UL16MiniAODAPVv2-106X_mcRun2_asymptotic_preVFP_v11-v1+MINIAODSIM
0
jobs_inf_2016_2017_2016_2017_QCD_Pt_80to120_TuneCP5_13TeV_pythia8+RunIISummer20UL16MiniAODAPVv2-106X_mcRun2_asymptotic_preVFP_v11-v1+MINIAODSIM
0
jobs_inf_2016_2017_2016_2017_QCD_Pt_470to600_TuneCP5_13TeV_pythia8+RunIISummer20UL16MiniAODv2-106X_mcRun2_asymptotic_v17-v1+MINIAODSIM
0
jobs_inf_2016_2017_2016_2017_QCD_HT2000toInf_TuneCP5_PSWeights_13TeV-madgraphMLM-pythia8+RunIISummer20UL16MiniAODAPVv2-106X_mcRun2_asymptotic_preVFP_v11-v1+MINIAODSIM
0
jobs_inf_2016_2017_2016_2017_QCD_Pt_470to600_TuneCP5_13TeV_pythia8+RunIISummer20UL16MiniAODAPVv2-106X_mcRun2_asymptotic_preVFP_v11-v1+MINIAODSIM
0
jobs_inf_2016_2017_2016_2017_QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8+RunIISummer20UL16MiniAODAP

grep: /work/submit/lavezzo/SUEP/logs/jobs_inf_2016_2017_2016_2017_QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8+RunIISummer20UL16MiniAODv2-106X_mcRun2_asymptotic_v17-v1+MINIAODSIM/*.*.err: No such file or directory
grep: /work/submit/lavezzo/SUEP/logs/jobs_inf_2016_2017_2016_2017_QCD_HT50to100_TuneCP5_PSWeights_13TeV-madgraphMLM-pythia8+RunIISummer20UL16MiniAODAPVv2-106X_mcRun2_asymptotic_preVFP_v11-v1+MINIAODSIM/*.*.err: No such file or directory
grep: /work/submit/lavezzo/SUEP/logs/jobs_inf_2016_2017_2016_2017_QCD_Pt_80to120_TuneCP5_13TeV_pythia8+RunIISummer20UL16MiniAODAPVv2-106X_mcRun2_asymptotic_preVFP_v11-v1+MINIAODSIM/*.*.err: No such file or directory
grep: /work/submit/lavezzo/SUEP/logs/jobs_inf_2016_2017_2016_2017_QCD_Pt_470to600_TuneCP5_13TeV_pythia8+RunIISummer20UL16MiniAODv2-106X_mcRun2_asymptotic_v17-v1+MINIAODSIM/*.*.err: No such file or directory
grep: /work/submit/lavezzo/SUEP/logs/jobs_inf_2016_2017_2016_2017_QCD_HT2000toInf_TuneCP5_PSWeights_13TeV-madgraphMLM-pythia8+RunII

In [8]:
bad_hosts = []
for dir_name in dirs:
    print(dir_name)
    
    error_files = os.popen('grep '+ err + ' ' + base_dir + dir_name +'/' + job_num + '.*').readlines()
    
    hosts = []
    for entry in error_files:
        if 'os.popen' in entry: continue
        current_file = entry.split('.err')[0]+'.out'
        f2 = open(current_file, "r")
        lines = f2.readlines()
        for line in lines:
            if 'Running on:' not in line: continue
            hostname = line[len("Running on:  "):-1]
            if hostname not in hosts: hosts.append(hostname)
    
    print(hosts, '\n')
    
    for host in hosts:
        if host not in bad_hosts: bad_hosts.append(host)

jobs_2018_data_V6_JetHT+Run2018B-UL2018_MiniAODv2-v1+MINIAOD
['t3btch042.mit.edu'] 



In [42]:
good_hosts = []
for dir_name in dirs:
    print(dir_name)
    
    error_files = os.popen('grep  Tau_jetIdx' + base_dir + dir_name +'/' + job_num + '.*').readlines()

    hosts = []
    for entry in error_files:
        if 'os.popen' in entry: continue
        current_file = entry.split('.err')[0]+'.out'
        f2 = open(current_file, "r")
        lines = f2.readlines()
        for line in lines:
            if 'Running on:' not in line: continue
            hostname = line[len("Running on:  "):-1]
            if hostname not in hosts: hosts.append(hostname)
        
    for host in hosts:
        if host not in good_hosts: good_hosts.append(host)

jobs_2018_MC_V6_QCD_Pt_170to300_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_2400to3200_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_30to50_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-pilot_106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_50to80_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_120to170_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v2+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8

In [54]:
print(bad_hosts)

['t2bat0270.cmsaf.mit.edu', 't2bat0256.cmsaf.mit.edu', 't2bat0465.cmsaf.mit.edu', 't2bat0500.cmsaf.mit.edu', 't2bat0461.cmsaf.mit.edu', 't2bat0490.cmsaf.mit.edu', 't2bat0498.cmsaf.mit.edu', 't2bat0497.cmsaf.mit.edu', 't2bat0202.cmsaf.mit.edu', 't2bat0485.cmsaf.mit.edu', 't2bat0079.cmsaf.mit.edu', 't2bat0486.cmsaf.mit.edu', 't2bat0494.cmsaf.mit.edu', 't2bat0240.cmsaf.mit.edu']


In [41]:
print(good_hosts)

[]


In [40]:
# is there any overlap between machines where the jobs
# that ran succesfully and those where they failed?
set(good_hosts).intersection(all_hosts)

set()